In [15]:
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

# Nuevas importaciones para la sintaxis moderna (LCEL)
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import  RunnablePassthrough
from pathlib import Path

In [16]:
# --- Carga de la API key ---
try:
    with open("../gem_apikey.txt") as f:
        api_key = f.read().strip()
    print("API key cargada correctamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'.")
    api_key = None

API key cargada correctamente.


In [17]:
def get_available_indexes(index_folder):
    """
    Busca en la carpeta de índices y devuelve una lista de los disponibles.
    """    
    if not os.path.exists(index_folder):
        print(f"Error: La carpeta de índices '{index_folder}' no fue encontrada.")
        return []

    # Obtener lista de directorios de índices (ej: 'faiss_index_doc1')
    processed_folders = [f.name for f in Path(index_folder).iterdir() if f.is_dir()]
    
    return processed_folders

In [18]:
def select_teacher(index_list):
    """
    Muestra un menú para seleccionar un profesor y devuelve los datos del profesor elegido.

    Args:
        index_list (list): Lista de índices de documentos disponibles.

    Returns:
        dict: Un diccionario con los datos del profesor seleccionado o None si no se elige uno válido.
    """
    teachers_data = [
        {"teacher": "Victor", "docs": 0},
        {"teacher": "Diego", "docs": len(index_list)},
        {"teacher": "Javier", "docs": 0},
        {"teacher": "Luis", "docs": 0},
    ]

    print("\n--- Profesores Disponibles ---")
    for i, item in enumerate(teachers_data):
        status = "✅ Disponible" if item["docs"] > 0 else "⛔ No disponible"
        print(f"{i + 1}. {item['teacher']} ({item['docs']} documentos) {status}")

    selected_index = -1
    while selected_index < 0 or selected_index >= len(teachers_data):
        try:
            choice = input(f"\nSelecciona un profesor (1-{len(teachers_data)}): ")
            selected_index = int(choice) - 1
            if selected_index < 0 or selected_index >= len(teachers_data):
                print("Selección inválida.")
        except ValueError:
            print("Entrada no válida.")

    selected_teacher = teachers_data[selected_index]

    if selected_teacher["docs"] == 0:
        print(f"\nEl profesor {selected_teacher['teacher']} aún no tiene temas disponibles.")
        return None

    print(f"\nHas seleccionado al profesor {selected_teacher['teacher']}.")
    return selected_teacher

In [19]:
def select_knowledge_area(index_list, base_index_folder):
    """
    Muestra un menú para seleccionar un área de conocimiento y devuelve sus detalles.

    Args:
        index_list (list): Lista de índices de documentos disponibles.
        base_index_folder (str): Ruta a la carpeta que contiene los índices.

    Returns:
        dict: Un diccionario con los detalles del área seleccionada o None.
    """
    print("\n--- Áreas de conocimiento disponibles para Consulta ---")
    friendly_names = [name.replace("faiss_index_", "") for name in index_list]
    for i, name in enumerate(friendly_names):
        print(f"{i + 1}. {name}")

    user_selection = -1
    while user_selection < 0 or user_selection >= len(index_list):
        try:
            choice = input(f"\nElige el área de conocimiento (1-{len(index_list)}): ")
            user_selection = int(choice) - 1
            if user_selection < 0 or user_selection >= len(index_list):
                print("Selección inválida.")
        except ValueError:
            print("Entrada no válida.")

    selected_folder_name = index_list[user_selection]
    selected_document_name = friendly_names[user_selection]
    full_index_path = os.path.join(base_index_folder, selected_folder_name)

    return {
        "folder_name": selected_folder_name,
        "document_name": selected_document_name,
        "full_path": full_index_path,
    }

In [20]:
def configure_rag_chain(api_key, teacher_name, knowledge_domain, index_path):
    """
    Configura y devuelve la cadena RAG completa de LangChain.

    Args:
        api_key (str): La API key de Google.
        teacher_name (str): Nombre del profesor para el prompt.
        knowledge_domain (str): Dominio de conocimiento para el prompt.
        index_path (str): Ruta al índice FAISS a cargar.

    Returns:
        Runnable: La cadena RAG ejecutable o None si ocurre un error.
    """
    print(f"\nCargando la base de datos de '{knowledge_domain}'...")
    try:
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
        vectorstore = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        retriever = vectorstore.as_retriever()
        print("¡Base de datos cargada exitosamente!")
    except Exception as e:
        print(f"Error al cargar la base de datos vectorial: {e}")
        return None

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, google_api_key=api_key)

    system_template = """Eres un asistente virtual que se comporta como un profesor de {knowledgeDomain} experto.
    Tu nombre es "IAsistente de {Teacher}". Eres amable, didáctico y te encanta {knowledgeDomain}. 
    REGLA ESTRICTA: Solo puedes responder preguntas relacionadas con {knowledgeDomain} basándote en el contexto proporcionado. Si la pregunta no está relacionada con estos temas, 
    responde: "Lo siento, mi especialidad es {knowledgeDomain}. No puedo responder preguntas sobre otros temas.
    Cuando te digan "salir", te despedirás amablemente."""
    
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_template = "{user_question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    chat_prompt_template = chat_prompt_template.partial(Teacher=teacher_name, knowledgeDomain=knowledge_domain)

    rag_chain = (
        {"context": retriever, "user_question": RunnablePassthrough()}
        | chat_prompt_template
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [21]:
def start_chat():
    """
    Orquesta el inicio del chat: selección, configuración y ejecución.
    """
    if not api_key or api_key == "TU_API_KEY_AQUI":
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- Configuración de Rutas ---
    FOLDER_BASE = "../database"
    FOLDER_INDEX = os.path.join(FOLDER_BASE, "indexed", "faiss")

    # --- Flujo del programa ---
    index_list = get_available_indexes(FOLDER_INDEX)
    if not index_list:
        print("No se encontraron documentos procesados para iniciar el chat.")
        return

    # 1. Seleccionar Profesor
    teacher_info = select_teacher(index_list)
    if not teacher_info:
        return # Termina si no se selecciona un profesor válido

    # 2. Seleccionar Área de Conocimiento
    area_info = select_knowledge_area(index_list, FOLDER_INDEX)
    if not area_info:
        return

    # 3. Configurar la cadena RAG
    rag_chain = configure_rag_chain(
        api_key=api_key,
        teacher_name=teacher_info["teacher"],
        knowledge_domain=area_info["document_name"],
        index_path=area_info["full_path"],
    )
    if not rag_chain:
        return

    # --- Inicio del Bucle de Chat ---
    print(f"\n¡El Asistente de {teacher_info['teacher']} está listo! Escribe 'salir' para terminar.")
    
    while True:
        query = input("\nPregunta del estudiante: ")
        if query.lower() == "salir":
            print(f"\nAsistente de {teacher_info['teacher']}: ¡Hasta luego! Espero haberte ayudado.")
            break
        
        response = rag_chain.invoke(query)
        print(f"\nRespuesta del Asistente de {teacher_info['teacher']}:")
        print(response)


if __name__ == "__main__":
    start_chat()


--- Profesores Disponibles ---
1. Victor (0 documentos) ⛔ No disponible
2. Diego (3 documentos) ✅ Disponible
3. Javier (0 documentos) ⛔ No disponible
4. Luis (0 documentos) ⛔ No disponible



Selecciona un profesor (1-4):  2



Has seleccionado al profesor Diego.

--- Áreas de conocimiento disponibles para Consulta ---
1. sistema-cardiovascular
2. sistema-nervioso-central
3. sistema-oseo



Elige el área de conocimiento (1-3):  4


Selección inválida.



Elige el área de conocimiento (1-3):  1



Cargando la base de datos de 'sistema-cardiovascular'...
¡Base de datos cargada exitosamente!

¡El Asistente de Diego está listo! Escribe 'salir' para terminar.



Pregunta del estudiante:  salir



Asistente de Diego: ¡Hasta luego! Espero haberte ayudado.
